In [2]:
!pip install jupyter-dash
!pip install dash==2.3.1
!pip install dash_bootstrap_components

# Library to Use Dash in Notebooks
from jupyter_dash import JupyterDash
# Dash HTML Components Library
from dash import html
# Dash Core Components Library
from dash import dcc
from dash.dependencies import Input, Output
# Plotly Express Library for Graphics
import plotly.express as px

import pandas as pd

import dash_bootstrap_components as dbc
from dash import dash_table

import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: dash
    Found existing installation: dash 2.17.0
    Uninstalling dash-2.17.0:
      Successfully uninstalled dash-2.17.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 2.6 MB/s eta 0:00:00


In [93]:
# Load the tips data
DATA_FILENAME = "tsdata_3.json"

with open(DATA_FILENAME) as fs:
  raw_data = json.load(fs)

metadata_dict = raw_data["channel"]
feeds_dict = raw_data["feeds"]
del raw_data

df = pd.DataFrame.from_dict(feeds_dict)

for column_name in ["field1", "field2", "field3"]:
  df[column_name]=pd.to_numeric(df[column_name], errors="coerce")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  161 non-null    object
 1   entry_id    161 non-null    int64 
 2   field1      161 non-null    int64 
 3   field2      161 non-null    int64 
 4   field3      161 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 6.4+ KB


In [94]:
# Some setup
def read_json():
    with open(DATA_FILENAME) as fs:
      raw_json = fs.read()
    return raw_json

In [95]:
# @title Texto de título predeterminado
# Instantiation
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.MORPH])

# Define a dropdown subclass to avoid repetition
class CustomDropdown(dcc.Dropdown):
  def __init__(self, *args, **kwargs):
    kwargs["options"] = [
                    {'label': 'Temperature', 'value': 'field1'},
                    {'label': 'Humidity', 'value': 'field2'},
                    {'label': 'Brightness', 'value': 'field3'}
                    ] # Si eso se añade mas aqui y ambos dropdowns lo añaden sin tener que tocar nada mas
    kwargs["value"] = "field1"
    super().__init__(*args, **kwargs)

# Tabs:
graph_tab = dbc.Card([
    dbc.CardBody(
        [
            CustomDropdown(id="dropdown_graph"),
            dcc.Graph(id='graph'),
            dcc.RangeSlider(0, 100, 10, value=[0, 100], id='slider_graph')]
        )
    ])

table_tab = dbc.Card([
    dbc.CardBody([
        CustomDropdown(id="dropdown_table"),
        dcc.RangeSlider(0, 100, 10, value=[0, 100], id='slider_table'),
        dash_table.DataTable(
            id='table',
            columns=[
                {'name': 'Entry Id', 'id': 'entry_id'},
                {'name': 'Value', 'id': "field1"}
            ],
            data=[]
            )
        ]
    )
])

json_tab = dbc.Card([
    dbc.CardBody([
        html.P(read_json())
        ]
    )
])

metadata_tab = dbc.Card([
    dbc.CardBody([
        html.P(str(metadata_dict))
        ]
    )
])

# Layout
app.layout = dbc.Container([
    html.H1('Data'),
    dbc.Row([
        dbc.Col([
            dbc.Tabs(children=[
                dbc.Tab(graph_tab, label="Graph View"),
                dbc.Tab(table_tab, label="Table View"),
                dbc.Tab(json_tab, label="Raw Json View"),
                dbc.Tab(metadata_tab, label = "Metadata View")]
                     )]
                )
        ])
])

# Callbacks
def mask_dataframe(df, range, fieldname):
  low, high = range
  return df[(df[fieldname] > low) & (df[fieldname] < high)]

@app.callback(
    Output(component_id='graph', component_property='figure'),
    Input(component_id='dropdown_graph', component_property='value'),
    Input(component_id='slider_graph', component_property='value')
)
def update_graph(dropdown_option, slider_range):
  curr_df = mask_dataframe(df, slider_range, dropdown_option)
  print(curr_df)
  graph = px.line(curr_df, x='entry_id', y=dropdown_option,
                 title="Tips based on several parameters",

                 labels={
                     "entry_id": "Entry",
                     "field1": "Temperature",
                     "field2": "Humidity",
                     "field3": "Brightness"
                 }
                     )
  return graph


@app.callback(
    Output(component_id='table', component_property='columns'),
    Output(component_id='table', component_property='data'),
    Input(component_id='dropdown_table', component_property='value'),
    Input(component_id='slider_table', component_property='value')
)
def update_table(dropdown_option, slider_range):
    curr_df = mask_dataframe(df, slider_range, dropdown_option)
    table_data = curr_df[['created_at', 'entry_id', dropdown_option]].to_dict("records")
    table_columns = [
        {"name": 'Timestamp', "id": 'created_at'},
        {'name': 'Entry Id', 'id': 'entry_id'},
        {'name': 'Value', 'id': dropdown_option}
     ]
    return table_columns, table_data



# Execution
app.run_server(debug=False, mode='external', port=8060)

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8060
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/May/2024 17:05:31] "GET /_alive_cec91189-2eb8-463d-8588-63e5f431c758 HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>